In [9]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
import xgboost
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import pandas as pd
import warnings
warnings.simplefilter(action = "ignore") 

In [10]:
df = pd.read_csv(r"C:\Users\Sadullah\data_science\diabetes.csv")

In [13]:
df['Outcome'].value_counts()*100/len(df['Outcome'])

0    65.104167
1    34.895833
Name: Outcome, dtype: float64

In [14]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [16]:
df[(df['Glucose']==0) | (df['BloodPressure']==0) | (df['SkinThickness']==0) | (df['Insulin']==0) | (df['BMI']==0)]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
5,5,116,74,0,0,25.6,0.201,30,0
7,10,115,0,0,0,35.3,0.134,29,0
...,...,...,...,...,...,...,...,...,...
761,9,170,74,31,0,44.0,0.403,43,1
762,9,89,62,0,0,22.5,0.142,33,0
764,2,122,70,27,0,36.8,0.340,27,0
766,1,126,60,0,0,30.1,0.349,47,1


In [17]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [18]:
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']]=df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [19]:
df.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [20]:
df.dropna(inplace=True)

In [21]:
df.shape

(392, 9)

In [22]:
df.describe([0.10,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90,0.95,0.99]).T

,count,mean,std,min,10%,20%,30%,40%,50%,60%,70%,80%,90%,95%,99%,max
Pregnancies,392.0,3.301020,3.211424,0.000,0.0000,1.0000,1.000,2.000,2.0000,3.000,4.0000,6.000,8.0000,10.00000,13.00000,17.00
Glucose,392.0,122.627551,30.860781,56.000,87.0000,95.0000,101.000,109.000,119.0000,126.000,136.0000,151.000,170.9000,181.00000,196.00000,198.00
BloodPressure,392.0,70.663265,12.496092,24.000,56.0000,60.0000,64.000,68.000,70.0000,74.000,77.4000,80.000,86.0000,90.00000,102.36000,110.00
SkinThickness,392.0,29.145408,10.516424,7.000,15.0000,19.0000,23.000,26.000,29.0000,32.000,35.0000,39.000,43.0000,46.45000,52.00000,63.00
Insulin,392.0,156.056122,118.841690,14.000,51.1000,70.0000,88.000,105.000,125.5000,147.200,176.0000,210.000,292.8000,396.50000,580.89000,846.00
BMI,392.0,33.086224,7.027659,18.200,24.4200,26.8200,29.300,30.940,33.2000,34.500,36.1700,38.200,42.0700,45.24500,53.36200,67.10
DiabetesPedigreeFunction,392.0,0.523046,0.345488,0.085,0.1834,0.2542,0.299,0.365,0.4495,0.527,0.6304,0.729,0.9422,1.16035,1.73842,2.42
Age,392.0,30.864796,10.200777,21.000,22.0000,23.0000,24.000,25.000,27.0000,29.000,33.0000,39.000,46.0000,52.45000,60.00000,81.00
Outcome,392.0,0.331633,0.471401,0.000,0.0000,0.0000,0.000,0.000,0.0000,0.000,1.0000,1.000,1.0000,1.00000,1.00000,1.00


In [23]:
for feature in df:
    
    Q1 = df[feature].quantile(0.10)
    Q3 = df[feature].quantile(0.90)
    IQR = Q3-Q1
    lower = Q1- 1.5*IQR
    upper = Q3 + 1.5*IQR
    
    if df[(df[feature] > upper) | (df[feature] < lower)].any(axis=None):
        print(feature,"yes")
        print(df[(df[feature] > upper) | (df[feature] < lower)].shape[0])
    else:
        print(feature, "no")

Pregnancies no
Glucose no
BloodPressure no
SkinThickness no
Insulin yes
3
BMI no
DiabetesPedigreeFunction yes
4
Age no
Outcome no


In [24]:
Q1 = df["SkinThickness"].quantile(0.10)
Q3 = df["SkinThickness"].quantile(0.90)
IQR = Q3-Q1
upper = Q3 + 1.5*IQR
lower = Q1 - 1.5*IQR
print(upper)
print(lower)

85.0
-27.0


In [25]:
df.loc[df["SkinThickness"] > upper, "SkinThickness"] = upper

In [26]:
Q1 = df["Insulin"].quantile(0.10)
Q3 = df["Insulin"].quantile(0.90)
IQR = Q3-Q1
upper = Q3 + 1.5*IQR
lower = Q1 - 1.5*IQR
print(upper)
print(lower)

655.3500000000001
-311.4500000000001


In [27]:
df[df["Insulin"] > upper]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
13,1,189.0,60.0,23.0,846.0,30.1,0.398,59,1
228,4,197.0,70.0,39.0,744.0,36.7,2.329,31,0
247,0,165.0,90.0,33.0,680.0,52.3,0.427,23,0


In [28]:
df.loc[df["Insulin"] > upper, "Insulin"] = upper

In [29]:
Q1 = df["DiabetesPedigreeFunction"].quantile(0.10)
Q3 = df["DiabetesPedigreeFunction"].quantile(0.90)
IQR = Q3-Q1
upper = Q3 + 1.5*IQR
lower = Q1 - 1.5*IQR
print(upper)
print(lower)

2.0804000000000014
-0.9548000000000008


In [30]:
df[df["DiabetesPedigreeFunction"] > upper]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
4,0,137.0,40.0,35.0,168.00,43.1,2.288,33,1
228,4,197.0,70.0,39.0,655.35,36.7,2.329,31,0
370,3,173.0,82.0,48.0,465.00,38.4,2.137,25,1
445,0,180.0,78.0,63.0,14.00,59.4,2.420,25,1


In [31]:
df.loc[df["DiabetesPedigreeFunction"] > upper, "DiabetesPedigreeFunction"] = upper

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 3 to 765
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               392 non-null    int64  
 1   Glucose                   392 non-null    float64
 2   BloodPressure             392 non-null    float64
 3   SkinThickness             392 non-null    float64
 4   Insulin                   392 non-null    float64
 5   BMI                       392 non-null    float64
 6   DiabetesPedigreeFunction  392 non-null    float64
 7   Age                       392 non-null    int64  
 8   Outcome                   392 non-null    int64  
dtypes: float64(6), int64(3)
memory usage: 30.6 KB


In [80]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
3,-0.25,-0.681818,-0.250,-0.3750,-0.278146,-0.586207,-0.677052,-0.461538,0.0
4,-0.50,0.409091,-1.875,0.3750,0.375276,1.137931,3.908688,0.461538,1.0
6,0.25,-0.931818,-1.250,0.1875,-0.331126,-0.252874,-0.482924,-0.076923,1.0
8,0.00,1.772727,0.000,1.0000,3.686534,-0.310345,-0.698622,2.000000,1.0
13,-0.25,1.590909,-0.625,-0.3750,4.678587,-0.356322,-0.123427,2.461538,1.0


In [34]:
y = df["Outcome"]
X = df.drop(["Outcome"], axis = 1)

In [35]:
num_features = list(df.select_dtypes(['int64','float64']).columns)

In [36]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler().fit(X)
df[num_features] = rs.fit_transform(df[num_features])

In [63]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('SVC', SVC(gamma='auto')))
models.append(("XGBoost", XGBClassifier()))
models.append(("LGBM", LGBMClassifier()))

In [64]:
X = df.drop("Outcome",axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=12345)


for name, model in models:
    
        mod = model.fit(X_train,y_train)
        y_pred = mod.predict(X_test)
        res = accuracy_score(y_test,y_pred)
        print(name+": "+str(res))

LR: 0.8607594936708861
KNN: 0.8860759493670886
CART: 0.7341772151898734
RF: 0.8481012658227848
SVC: 0.8227848101265823
XGBoost: 0.8481012658227848
LGBM: 0.810126582278481


In [39]:
y_test.value_counts()

0.0    59
1.0    20
Name: Outcome, dtype: int64

# KNN MODEL & HYPERPARAMETER OPTİMİZATİON #

In [40]:
KNN = KNeighborsClassifier().fit(X_train,y_train)
y_pred = KNN.predict(X_test)
res = accuracy_score(y_test,y_pred)
print(res)

0.8860759493670886


In [41]:
for k in range(30):
    k = k + 2
    knn_model = KNeighborsClassifier(n_neighbors = k).fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)
    res = accuracy_score(y_test,y_pred)
    print("k =", k, "için accuracy değeri:", res)

k = 2 için accuracy değeri: 0.7721518987341772
k = 3 için accuracy değeri: 0.8607594936708861
k = 4 için accuracy değeri: 0.8607594936708861
k = 5 için accuracy değeri: 0.8860759493670886
k = 6 için accuracy değeri: 0.8481012658227848
k = 7 için accuracy değeri: 0.8354430379746836
k = 8 için accuracy değeri: 0.8481012658227848
k = 9 için accuracy değeri: 0.8607594936708861
k = 10 için accuracy değeri: 0.8734177215189873
k = 11 için accuracy değeri: 0.8607594936708861
k = 12 için accuracy değeri: 0.8481012658227848
k = 13 için accuracy değeri: 0.8607594936708861
k = 14 için accuracy değeri: 0.8481012658227848
k = 15 için accuracy değeri: 0.810126582278481
k = 16 için accuracy değeri: 0.8481012658227848
k = 17 için accuracy değeri: 0.8481012658227848
k = 18 için accuracy değeri: 0.8734177215189873
k = 19 için accuracy değeri: 0.8607594936708861
k = 20 için accuracy değeri: 0.8481012658227848
k = 21 için accuracy değeri: 0.8481012658227848
k = 22 için accuracy değeri: 0.8607594936708861
k

#  CART Model ve Hiperparametre Optimizasyonu # 

In [42]:
cart_model = DecisionTreeClassifier().fit(X_train,y_train)
y_pred = cart_model.predict(X_test)
res = accuracy_score(y_test,y_pred)
print(res)

0.7468354430379747


In [43]:
max_depth_values = [2,3,4,5,6,7,8,9]
min_samples_split_values = [2,3,4,5,6,7,8,9]

In [44]:
for i in range(10,1001,50):
    max_depth_values.append(i)
for j in range(10,101,10):
    min_samples_split_values.append(j)

In [45]:
print(max_depth_values)
print(min_samples_split_values)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 60, 110, 160, 210, 260, 310, 360, 410, 460, 510, 560, 610, 660, 710, 760, 810, 860, 910, 960]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]


In [46]:
cart_params = {"max_depth": max_depth_values,
              "min_samples_split": min_samples_split_values}

In [47]:
cart_model = DecisionTreeClassifier()
cart_cv_model = GridSearchCV(cart_model, cart_params, cv = 10).fit(X_train, y_train)
cart_cv_model.best_params_
cart_tuned = DecisionTreeClassifier(**cart_cv_model.best_params_).fit(X_train, y_train)

In [48]:
cart_cv_model.best_params_

{'max_depth': 3, 'min_samples_split': 5}

In [49]:
cart_model = DecisionTreeClassifier(max_depth = 3, min_samples_split = 4).fit(X_train,y_train)
y_pred = cart_model.predict(X_test)
res = accuracy_score(y_test,y_pred)
print(res)

0.7974683544303798


# Random Forests Model ve Hiperparametre Optimizasyonu # 

In [50]:
rf_model = RandomForestClassifier().fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
res = accuracy_score(y_test,y_pred)
print(res)

0.7974683544303798


In [51]:
max_depth_values = [5,8,10,None]
max_features_values = [2,5,10,15,17]
n_estimators_values = [100,200,500,1000]
min_samples_split_values = [2,5,10,20,30]


In [52]:
print(max_depth_values)
print(max_features_values)
print(n_estimators_values)
print(min_samples_split_values)

[5, 8, 10, None]
[2, 5, 10, 15, 17]
[100, 200, 500, 1000]
[2, 5, 10, 20, 30]


In [53]:
rf_params = {"max_depth": max_depth_values,
            "max_features": max_features_values,
            "n_estimators": n_estimators_values,
            "min_samples_split": min_samples_split_values}

In [54]:
rf_cv_model = GridSearchCV(rf_model, rf_params, cv = 10, n_jobs = -1, verbose = 2).fit(X_train, y_train)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 22.8min
[Parallel(n_jobs=-1)]: Done 4000 out of 4000 | elapsed: 26.1min finished


In [55]:
rf_cv_model.best_params_

{'max_depth': 5,
 'max_features': 2,
 'min_samples_split': 30,
 'n_estimators': 500}

In [58]:
rf_tuned = RandomForestClassifier(**rf_cv_model.best_params_).fit(X_train, y_train)

In [59]:
rf_model = RandomForestClassifier(max_depth = 5, min_samples_split = 30, max_features= 2, n_estimators= 500).fit(X_train,y_train)
y_pred = rf_model.predict(X_test)
res = accuracy_score(y_test,y_pred)
print(res)

0.810126582278481


## Light GBM

In [65]:
lgbm=LGBMClassifier().fit(X_train,y_train)
y_pred=lgbm.predict(X_test)
res=accuracy_score(y_test,y_pred)
print(res)

0.810126582278481


In [66]:
lgbm_params = {"learning_rate": [0.01,0.001, 0.1, 0.5, 1],
              "n_estimators": [200,500,1000,5000],
              "max_depth": [2,4,6,7,10],
              "colsample_bytree": [1,0.8,0.5,0.4]}

In [67]:
lgbm_cv_model = GridSearchCV(lgbm,
                             lgbm_params,
                             cv = 5,
                             n_jobs = -1,
                             verbose =2).fit(X_train, y_train)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   50.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  7.7min finished


In [68]:
lgbm_cv_model.best_params_

{'colsample_bytree': 1,
 'learning_rate': 0.01,
 'max_depth': 7,
 'n_estimators': 200}

In [70]:
lgbm_tuned = LGBMClassifier(**lgbm_cv_model.best_params_).fit(X_train, y_train)

In [72]:
lgbm_model = LGBMClassifier(max_depth = 7 , colsample_bytree = 1, learning_rate= 0.01, n_estimators= 200).fit(X_train,y_train)
y_pred = lgbm_model.predict(X_test)
res = accuracy_score(y_test,y_pred)
print(res)

0.8607594936708861


## XGBOOST

In [79]:
xgbm=XGBClassifier().fit(X_train,y_train)
y_pred=xgbm.predict(X_test)
res=accuracy_score(y_test,y_pred)
print(res)

0.8481012658227848


In [74]:
xgbm_params = {"learning_rate": [0.1,0.01,0.5],
             "max_depth": [2,3,4,5,8],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree": [0.4,0.7,1]}

In [75]:
xgbm_cv_model = GridSearchCV(xgbm,
                             xgbm_params,
                             cv = 5,
                             n_jobs = -1,
                             verbose =2).fit(X_train, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   25.7s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  1.1min finished


In [76]:
xgbm_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.01,
 'max_depth': 2,
 'n_estimators': 500}

In [77]:
xgbm_tuned = XGBClassifier(**xgbm_cv_model.best_params_).fit(X_train, y_train)

In [78]:
xgbm_model = XGBClassifier(max_depth = 2 , colsample_bytree = 0.4, learning_rate= 0.01, n_estimators= 500).fit(X_train,y_train)
y_pred = lgbm_model.predict(X_test)
res = accuracy_score(y_test,y_pred)
print(res)

0.8607594936708861


# Sonuçlar

## Hiperparametre Optimizasyonu Yapılmamış Modeller

In [ ]:
LR: 0.8607594936708861
KNN: 0.8860759493670886
CART: 0.7341772151898734
RF: 0.8481012658227848
SVC: 0.8227848101265823
XGBoost: 0.8481012658227848
LGBM: 0.810126582278481

## Hiperparametre Optimizasyonu Yapılmış Modeller

In [82]:
KNN: 0.8860759493670886
CART: 0.7974683544303798
RF: 0.810126582278481
LGBM: 0.8607594936708861
XGB: 0.8607594936708861